# Modeling Source-to-Sink systems using FastScape: 7. Piedmont deposition

![Lannemezan Fan](LannemezanFan.jpg "Lannemezan Fan")

In [ ]:
import xsimlab as xs
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
#plt.style.use('dark_background')
%load_ext xsimlab.ipython
import hvplot.xarray

## sediment_model

In FastScape, we have included a complete sediment model, i.e., that includes the solution of the equation given in the previous notebook:

$$\frac{\partial h}{\partial t}=U-K_fA^mS^n + \frac{Gq_s}{A} + k_d\nabla^2h$$

For the purpose of studying the depositional system, we will neglect the effect of diffusion.

In [ ]:
from fastscape.models import sediment_model

transit_model = sediment_model.drop_processes('diffusion')

transit_model.visualize(show_variables=True)

We would like to create a constant flux boundary condition which does not exist in FastScape at this stage. To *mimic* such a boundary condition, we will create a small (1 pixel wide) region in constant uplift along one of the boundaries. The opposite boundary will be assumed to be at base level. The other two, for simplicity, will be cyclic (looped).

The flux will be given by the product of the uplift rate by the area of a given pixel.

In this first simple experiment, we will assume that the rest of the model does not experience uplift/subsidence. It could be compared to the formation of a piedmont deposit.

In [ ]:
xl = 100e3
yl = 100e3
nx = 101
ny = 101
X = np.linspace(0,xl,nx)
Y = np.linspace(0,yl,ny)
x, y = np.meshgrid(X, Y)

u0 = 1e-3
u = np.zeros((ny,nx))
ylim = 2*yl/(ny-1)
u = np.where(y<ylim, u0, 0)


In [ ]:
# %create_setup transit_model --default --verbose
import xsimlab as xs

ds_in = xs.create_setup(
    model=transit_model,
    clocks={'time': np.linspace(0,1e7,101)},
    master_clock='time',
    input_vars={
        # nb. of grid nodes in (y, x)
        'grid__shape': [ny,nx],
        # total grid length in (y, x)
        'grid__length': [yl,xl],
        # node status at borders
        'boundary__status': ['looped', 'looped', 'fixed_value', 'core'],
        # uplift rate
        'uplift__rate': u,
        # random seed
        'init_topography__seed': None,
        # MFD partioner slope exponent
        'flow__slope_exp': 1,
        # drainage area exponent
        'spl__area_exp': 0.4,
        # slope exponent
        'spl__slope_exp': 1,
        # bedrock channel incision coefficient
        'spl__k_coef_bedrock': 1e-5,
        # soil (sediment) channel incision coefficient
        'spl__k_coef_soil': 1e-5,
        # detached bedrock transport/deposition coefficient
        'spl__g_coef_bedrock': 1,
        # soil (sediment) transport/deposition coefficient
        'spl__g_coef_soil': 1
    },
    output_vars={'topography__elevation': 'time',
                'drainage__area': 'time'}
)


In [ ]:
with xs.monitoring.ProgressBar():
    ds_out = ds_in.xsimlab.run(model=transit_model)

Let's look at the solution

In [ ]:
from ipyfastscape import TopoViz3d

app = TopoViz3d(ds_out, canvas_height=600, time_dim="time")

app.components['background_color'].set_color('black')
app.components['vertical_exaggeration'].set_factor(5)
app.components['timestepper'].go_to_time(ds_out.time[-1])

app.show()

We can also draw a section across the model, starting at the limit of the uplifting region:

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, sharex=False, sharey=True, figsize=(12,7))

ds_out.topography__elevation.isel(time=-1).sel(x=xl/2)[ds_out.y>ylim].plot()


We can also visualize the evoluton of the topography in the piedmont (averaged i the *x*=direction)

In [ ]:
ds_out.topography__elevation.mean('x').hvplot(x='y',
                                          cmap=plt.cm.viridis,
                                          groupby='time', width=600)

Now analyze the results by producing various cross-sections.